<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.61
    Uninstalling yfinance-0.2.61:
      Successfully uninstalled yfinance-0.2.61
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))
  print('-------------------')
  print('Est LTT: ',  format_amt(estimate_lt))
  print('Est LTT PnL: '+ '{} ({}%)'.format(format_amt(est_lt_pnl_amount), est_lt_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')

Run date time (IST): 2025-05-30 15:01:42
-------------------
qualified stocks: 93
with latest results: 32
still star stocks: 18
-------------------
Initial Investment:  1.17 C
CY Investment:  1.44 C
Reserve:  4.24 L
Current:  1.32 C
-------------------
Today PnL: 1.87 K (0.01%)
Current PnL: -16.19 L (-11.28%)
CY Booked + Current PnL: -11.71 L (-8.16%)
-------------------
Total profit:  4.05 L
Total loss:  -20.24 L
-------------------
Total Booked + Current PnL: 15.27 L (13.1%)
Total Booked PnL: 31.45 L (26.99%)
Curr Year Booked PnL: 4.47 L (3.38%)
Prev Year Booked PnL: 26.98 L (23.15%)
Est FTT:  2.12 C
Est FTT PnL: 79.34 L (60.01%)
-------------------
Est LTT:  2.73 C
Est LTT PnL: 1.41 C (106.36%)
Deployed:  1.17 C
Current:  1.32 C
CAGR/XIRR %: 9.09%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
49,LICI,785.05,983.0,-2.96,H-LC,92.47,237324.0,42632.0,6455.0,1.32,21.90,2.72,25.21,29.0,6.60,1.85,31.23,XY25,NTT,INSURANCE
4,ADANIPORTS,1103.69,1583.0,1.42,M-LC,87.10,101601.0,23239.0,10790.0,-0.53,29.66,10.62,43.43,71.0,2.15,0.79,35.89,XY24,NTT,MISC
62,RELIANCE,1291.83,1526.0,3.85,X-LC,27.96,154540.0,13731.0,11791.0,0.35,9.75,7.63,18.13,17.0,1.16,1.21,22.02,XY25,NTT,REFINERIES
68,SBIN,760.30,863.0,-12.51,M-LC,48.39,214570.0,13090.0,14119.0,1.55,6.50,6.58,13.51,73.0,0.93,1.68,19.91,XY25,NTT,BANKS
56,PIDILITIND,2504.06,3576.0,-16.01,X-LC,34.41,93084.0,17962.0,14195.0,2.42,23.91,15.25,42.81,11.0,1.27,0.73,16.74,X40,BTT,CHEMICALS


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,ASIANTILES,75.41,137.0,6694.44,M-SC,98.92,74540.0,-19270.0,95888.0,12.31,-20.54,128.64,81.67,203.0,-0.20,0.58,44.35,XR,NTT,CERAMICS
23,DEN,52.79,75.0,117.62,L-SC,86.02,60864.0,-23600.0,59135.0,7.55,-27.94,97.16,42.07,235.0,-0.40,0.48,29.26,AR,NTT,ENTERTAINMENT
85,UNIONBANK,123.87,163.0,-4.91,M-LC,83.87,166775.0,25935.0,18562.0,3.95,18.41,11.13,31.59,76.0,1.40,1.30,44.87,XY24,NTT,BANKS
9,AWL,342.88,485.0,-67.84,H-MC,35.48,174117.0,-43612.0,133861.0,3.88,-20.03,76.88,41.45,86.0,-0.33,1.36,16.31,XY24,NTT,FMCG
16,CAMPUS,294.86,393.0,NaN,NaN,93.55,167212.0,-563.0,56401.0,2.99,-0.34,33.73,33.28,175.0,-0.01,1.31,29.46,XY24,NTT,FOOTWEAR


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
70,SHALBY,261.39,327.00,853.75,M-SC,15.05,129660.0,-51483.0,96947.0,-5.04,-28.42,74.77,25.10,204.0,-0.53,1.01,3.78,XY24,NTT,HEALTHCARE
15,BSOFT,628.85,844.81,-25.25,M-SC,39.78,70184.0,-39865.0,77659.0,-4.68,-36.22,110.65,34.34,171.0,-0.51,0.55,15.54,XR,ATH,IT
20,COFFEEDAY,59.14,80.00,-59.76,L-SC,13.98,60864.0,-52685.0,92738.0,-4.37,-46.40,152.37,35.27,238.0,-0.57,0.48,48.97,XR,NTT,HOTELS
89,VIPIND,488.80,718.00,-830.70,H-SC,82.80,69644.0,-25672.0,70368.0,-3.13,-26.93,101.04,46.89,132.0,-0.36,0.54,37.37,X40N,NTT,MISC
29,GREENPANEL,375.16,537.00,179.07,M-SC,63.44,124474.0,-58604.0,137581.0,-2.53,-32.01,110.53,43.14,189.0,-0.43,0.97,17.09,XY24,NTT,MISC


In [12]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
41,INFY,1450.31,2275.00,-12.78,X-LC,50.54,249010.0,18411.0,112727.0,0.16,7.98,45.27,56.86,1.0,0.16,1.95,15.25,X40,BTT,IT
80,TCS,3841.06,4998.00,-17.12,X-LC,24.73,239057.0,-25976.0,105807.0,-0.96,-9.80,44.26,30.12,2.0,-0.25,1.87,7.21,X40,BTT,IT
50,LTIM,5564.16,7197.33,-5.11,X-LC,73.12,182880.0,-17430.0,76224.0,-0.94,-8.70,41.68,29.35,4.0,-0.23,1.43,26.34,X200,ATH,IT
71,SIEMENS,4406.90,7969.85,-9.25,X-LC,76.34,82048.0,-28124.0,117197.0,-2.01,-25.53,142.84,80.85,5.0,-0.24,0.64,20.84,AR,ATH,ELECTRICAL
32,HAVELLS,1588.02,2069.16,-7.74,X-LC,8.60,238571.0,-9160.0,84216.0,-1.38,-3.70,35.30,30.30,6.0,-0.11,1.86,9.10,X40,ATH,ELECTRICAL


In [13]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,PFIZER,4245.50,6318.3,12.48,H-SC,97.85,129860.0,32214.0,15466.0,1.09,32.99,11.91,48.82,138.0,2.08,1.01,44.26,X40,ATH,PHARMA
4,ADANIPORTS,1103.69,1583.0,1.42,M-LC,87.10,101601.0,23239.0,10790.0,-0.53,29.66,10.62,43.43,71.0,2.15,0.79,35.89,XY24,NTT,MISC
49,LICI,785.05,983.0,-2.96,H-LC,92.47,237324.0,42632.0,6455.0,1.32,21.90,2.72,25.21,29.0,6.60,1.85,31.23,XY25,NTT,INSURANCE
56,PIDILITIND,2504.06,3576.0,-16.01,X-LC,34.41,93084.0,17962.0,14195.0,2.42,23.91,15.25,42.81,11.0,1.27,0.73,16.74,X40,BTT,CHEMICALS


In [14]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,LUXIND,1448.19,2442.00,35.79,M-SC,58.06,140790.0,3212.0,91204.0,-2.20,2.33,64.78,68.62,182.0,0.04,1.10,22.06,X40N,NTT,TEXTILES
55,PFIZER,4245.50,6318.30,12.48,H-SC,97.85,129860.0,32214.0,15466.0,1.09,32.99,11.91,48.82,138.0,2.08,1.01,44.26,X40,ATH,PHARMA
28,GILLETTE,8109.44,10330.69,8.65,H-SC,84.95,160225.0,22365.0,15398.0,1.18,16.22,9.61,27.39,118.0,1.45,1.25,34.06,X40,ATH,FMCG
57,POLYCAB,5005.95,7541.35,7.21,H-MC,59.14,101890.0,16789.0,26318.0,-0.15,19.73,25.83,50.65,92.0,0.64,0.80,28.08,X40N,ATH,CABLES
4,ADANIPORTS,1103.69,1583.00,1.42,M-LC,87.10,101601.0,23239.0,10790.0,-0.53,29.66,10.62,43.43,71.0,2.15,0.79,35.89,XY24,NTT,MISC


In [15]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
17,CAMS,3643.00,5250.99,-0.89,H-SC,33.33,111320.0,9316.0,35711.0,0.08,9.13,32.08,44.14,111.0,0.26,0.87,26.86,XR,ATH,MISC
11,BANKINDIA,113.49,190.00,-29.44,M-MC,52.69,139120.0,9968.0,77100.0,2.55,7.72,55.42,67.42,155.0,0.13,1.09,35.31,XR,NTT,BANKS
92,WIPRO,243.46,420.00,-9.71,M-LC,44.09,155000.0,4055.0,105400.0,-0.06,2.69,68.00,72.51,69.0,0.04,1.21,16.70,XR,NTT,IT
33,HEROMOTOCO,4311.81,6039.03,-0.48,H-MC,77.42,151781.0,868.0,59589.0,-0.46,0.57,39.26,40.06,89.0,0.01,1.19,24.15,AR,ATH,AUTO
38,INDIAMART,2327.09,4911.36,-52.59,H-SC,30.11,123686.0,350.0,136611.0,0.33,0.28,110.45,111.05,108.0,0.00,0.97,21.19,AR,ATH,MISC


In [16]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
20,COFFEEDAY,59.14,80.0,-59.76,L-SC,13.98,60864.0,-52685.0,92738.0,-4.37,-46.40,152.37,35.27,238.0,-0.57,0.48,48.97,XR,NTT,HOTELS
52,MASFIN,326.60,399.5,-22.31,H-SC,64.52,88455.0,-9525.0,31393.0,1.11,-9.72,35.49,22.32,130.0,-0.30,0.69,28.29,XR,ATH,FINANCE
4,ADANIPORTS,1103.69,1583.0,1.42,M-LC,87.10,101601.0,23239.0,10790.0,-0.53,29.66,10.62,43.43,71.0,2.15,0.79,35.89,XY24,NTT,MISC
84,UJJIVANSFB,52.77,60.0,34.19,M-SC,29.03,116343.0,-26136.0,45653.0,-1.06,-18.34,39.24,13.70,217.0,-0.57,0.91,37.32,X40N,NTT,BANKS
87,VALIANTORG,512.64,838.0,-383.22,H-SC,100.00,120289.0,-12997.0,97590.0,1.39,-9.75,81.13,63.47,131.0,-0.13,0.94,101.63,XR,NTT,CHEMICALS


In [17]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
87,VALIANTORG,512.64,838.00,-383.22,H-SC,100.00,120289.0,-12997.0,97590.0,1.39,-9.75,81.13,63.47,131.0,-0.13,0.94,101.63,XR,NTT,CHEMICALS
7,ASIANTILES,75.41,137.00,6694.44,M-SC,98.92,74540.0,-19270.0,95888.0,12.31,-20.54,128.64,81.67,203.0,-0.20,0.58,44.35,XR,NTT,CERAMICS
55,PFIZER,4245.50,6318.30,12.48,H-SC,97.85,129860.0,32214.0,15466.0,1.09,32.99,11.91,48.82,138.0,2.08,1.01,44.26,X40,ATH,PHARMA
77,TANLA,917.30,1963.11,-34.59,H-SC,96.77,160606.0,-78809.0,351759.0,1.53,-32.92,219.02,114.01,110.0,-0.22,1.25,47.30,AR,ATH,IT
74,STARHEALTH,551.56,761.00,6.12,M-SC,95.70,173460.0,-26756.0,102775.0,-0.19,-13.36,59.25,37.97,194.0,-0.26,1.36,39.27,XY24,NTT,INSURANCE


In [18]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.86
1,25,41.25
2,50,70.11


In [19]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24    32.13
XY25    14.39
X40     14.37
X40N    13.49
AR      12.46
XR      10.59
X200     1.43
SR       1.15
Name: CurrAlloc%, dtype: float64

In [20]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    21.51
M-SC    17.84
X-LC    17.35
H-MC     8.81
M-LC     7.84
H-LC     7.27
X-MC     5.62
M-MC     5.57
X-SC     2.82
L-SC     2.38
L-LC     1.15
Name: CurrAlloc%, dtype: float64

In [21]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,13.43,-0.69,32.38
IT,9.95,-17.74,82.10
BANKS,9.82,-11.65,62.98
MISC,6.42,-31.38,89.49
FINANCE,5.93,-25.63,85.28
INSURANCE,5.46,4.66,23.91
PAINTS,5.42,-18.68,53.80
ELECTRICAL,4.99,-4.99,56.47
AUTO,4.49,-8.99,56.60


In [22]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2862919.0,26
AR,1254850.0,12
X40N,1223534.0,17
XR,1097218.0,14
X40,761675.0,11
XY25,459491.0,10
SR,198127.0,2
X200,76224.0,1


In [23]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2264076.0,21
M-SC,2034905.0,21
X-LC,799656.0,12
M-MC,554612.0,5
H-MC,500778.0,8
X-SC,349347.0,4
X-MC,313483.0,5
M-LC,301849.0,6
L-SC,294131.0,4


In [24]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      938986.0      6
M-SC       XY24      746888.0      6
H-SC       AR        547814.0      3
           X40N      502031.0      6
M-SC       X40N      478366.0      6
X-LC       X40       452115.0      5
M-MC       XY24      396484.0      3
M-SC       XR        288563.0      4
H-LC       AR        269759.0      3
H-SC       XR        244381.0      4
M-SC       X40       220663.0      2
M-LC       XY24      182330.0      4
H-MC       XY24      172004.0      2
L-SC       XR        154497.0      2
X-LC       AR        143523.0      2
X-MC       XY24      142759.0      2
X-LC       XY25      127794.0      4
X-SC       X40N      121521.0      2
M-SC       XY25      120743.0      1
X-SC       XR        115379.0      1
           SR        112447.0      1
H-MC       XR        111898.0      1
M-LC       XR        105400.0      1
X-MC       X40N       95298.0      2
M-SC       AR         94002.0      1
           SR         85680.0      1
M-MC       AR         81028.0      1
L-SC       XY24       80499.0      1
M-MC       XR         77100.0      1
X-LC       X200       76224.0      1
X-MC       XY25       75426.0      1
H-MC       XY25       72936.0      1
           AR         59589.0      1
L-SC       AR         59135.0      1
H-MC       X40        58033.0      2
L-LC       XY25       42018.0      1
H-SC       X40        30864.0      2
H-MC       X40N       26318.0      1
M-LC       XY25       14119.0      1
H-LC       XY25        6455.0      1

In [25]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 35.0 seconds
